Imports libraries

In [1]:
import numpy as np
import pandas as pd
from pandas import *
import re
import math

Import files  -- excel

In [2]:
file = ('segundoejercicio.xlsx')
file2 = ('butacas.xlsx')
repository_data = pd.read_excel(file, skiprows= 6, usecols=[0,1,2,3,4,5])
repository_butacas = pd.read_excel(file2)

Transform Data

In [3]:

filter_valid_rows = repository_data.dropna(how='all')
filter_butacas = repository_butacas.dropna(how='all')

Iterate data, and transform to correct format

In [4]:
last_screen = 0
last_cinema = 0
remove_index_list = []
for index, row in filter_valid_rows.iterrows():
    #change NAN value
    try:
        screen = float( row['Screen'] )
        if( math.isnan( screen ) ):
            filter_valid_rows.at[index,'Screen'] = last_screen
        else:
            last_screen = screen

    except:
        remove_index_list = remove_index_list + [index]
   
    # remove cinema name. 
    #change NAN
    try:
        #remove text
        cinema = re.findall('[0-9]+', row['Cinema'])
        last_cinema = (cinema[0])
        filter_valid_rows.at[index,'Cinema'] = last_cinema
        
    except:
        filter_valid_rows.at[index,'Cinema'] = last_cinema
    
    if 'Total' == row['Film']:
        remove_index_list = remove_index_list + [index]
  

Remove Totals

In [5]:
filter_valid_rows = filter_valid_rows.drop(filter_valid_rows.index[remove_index_list])
   

Reduce results

In [6]:
for index, row in filter_valid_rows.iterrows():
    film_index = ( str(row['Film']) ).find('(')
    filter_valid_rows.at[index,'Film'] = (str(row['Film'])[0:film_index])

In [7]:
filter_valid_rows

,Cinema,Screen,Film,Number of Sessions,Admits,Cin Box Office Gross
0,2200,1,Batman,48.0,1830.0,1.580150e+05
1,2200,1.0,Batman,18.0,419.0,3.709200e+04
2,2200,1.0,Muestrame al Padre,1.0,84.0,5.040000e+03
3,2200,1.0,Los tipos Malos,5.0,51.0,3.742000e+03
4,2200,1.0,Morbius,2.0,37.0,3.773000e+03
...,...,...,...,...,...,...
1389,795,10.0,El Exorcismo,5.0,46.0,1.744700e+02
1390,795,10.0,Casate Conmigo,4.0,37.0,1.137100e+02
1391,795,10.0,Morbius,3.0,16.0,1.276900e+02
1392,795,10.0,Uncharted fuera del mapa,1.0,6.0,1.998000e+01
